Project 3 Part 1 (Core)


Business Problem
For this project, you have been hired to produce a MySQL database on Movies from a subset of IMDB's publicly available dataset. Ultimately, you will use this database to analyze what makes a movie successful and will provide recommendations to the stakeholder on how to make a successful movie.

In [1]:
# IMPORT THE PACKAGE

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
!pip install tmdbsimple
import os, json, math, time
from tqdm.notebook import tqdm_notebook

In [2]:
# upload the files
basics = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz', 
                     sep='\t', low_memory=False)

In [3]:
ratings = pd.read_csv('https://datasets.imdbws.com/title.ratings.tsv.gz', 
                      sep='\t', low_memory=False)

In [4]:
akas = pd.read_csv('https://datasets.imdbws.com/title.akas.tsv.gz', 
                   sep='\t', low_memory=False)

In [5]:
# replace the bad spelling missing values with Nan
basics = basics.replace({'\\N':np.nan}) 
ratings = ratings.replace({'\\N':np.nan}) 
akas = akas.replace({'\\N':np.nan}) 

In [6]:
# getting just the movies
basics = basics.loc[basics['titleType'] == 'movie']

In [7]:
# deliting the missing values
basics = basics.dropna(subset = ['runtimeMinutes', 'genres','startYear'])

In [8]:
# loc the dates the request from us 
basics['startYear'] = basics['startYear'].astype(int)
basics = basics.loc[(basics['startYear'] >= 2000) 
                    & (basics['startYear'] <= 2022)]

In [9]:
# taking out the documentary from our data
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]

In [10]:
# checking the values
basics['genres'].unique()

array(['Comedy,Fantasy,Romance', 'Drama', 'Drama,War',
       'Comedy,Horror,Sci-Fi', 'Comedy', 'Horror,Music,Thriller',
       'Comedy,Drama,Fantasy', 'Drama,Romance', 'Comedy,Mystery',
       'Drama,Fantasy', 'Adventure', 'Musical,Romance',
       'Action,Adventure,Drama', 'Action', 'Crime,Thriller',
       'Comedy,Fantasy', 'Action,Crime,Drama', 'Action,Thriller',
       'Comedy,Drama,Romance', 'Drama,Music,Romance',
       'Comedy,Horror,Mystery', 'Crime,Drama,Thriller', 'Comedy,Drama',
       'Action,Adventure,Animation', 'Comedy,Romance', 'Drama,Thriller',
       'Comedy,Drama,Sci-Fi', 'Adventure,Family,Fantasy', 'Drama,History',
       'Drama,History,War', 'Adventure,Animation,Comedy',
       'Action,Adventure,Fantasy', 'Action,Drama,Sci-Fi',
       'Biography,Drama,Romance', 'Horror,Mystery,Thriller',
       'Comedy,Drama,Thriller', 'Animation,Family,Musical',
       'Drama,Mystery,Thriller', 'Action,Adventure,Thriller',
       'Action,Horror,Sci-Fi', 'Action,Adventure,Sci-Fi',

In [11]:
# just us movies
akas = akas.loc[akas['region'] == 'US']

In [12]:
keepers = basics['tconst'].isin(akas['titleId'])

In [13]:
basics= basics[keepers]

In [14]:
# loading to our file
import os
os.makedirs('movies/',exist_ok=True) 
# Confirm folder created
os.listdir("movies/")

['.ipynb_checkpoints',
 'akas.csv.gz',
 'basics.csv.gz',
 'final2001.csv.gz',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'ratings.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json']

In [15]:
# loading to our file
basics.to_csv('movies/basics.csv.gz',compression='gzip',index=False)
ratings.to_csv('movies/ratings.csv.gz',compression='gzip',index=False)
akas.to_csv('movies/akas.csv.gz',compression='gzip',index=False)

In [16]:
basics = pd.read_csv("movies/basics.csv.gz", low_memory = False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0093119,movie,Grizzly II: Revenge,Grizzly II: The Predator,0,2020,NaN,74,"Horror,Music,Thriller"


In [17]:
import json
with open('C:/Users/raged/.secret/tmbd_api.json') as f: #change the path to match YOUR path!!
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

In [18]:
import tmdbsimple as tmbd
tmbd.API_KEY= login['api-key']

In [19]:
movie= tmbd.Movies(601)

info= movie.info()
info

{'adult': False,
 'backdrop_path': '/mXLVA0YL6tcXi6SJSuAh9ONXFj5.jpg',
 'belongs_to_collection': None,
 'budget': 10500000,
 'genres': [{'id': 878, 'name': 'Science Fiction'},
  {'id': 12, 'name': 'Adventure'},
  {'id': 10751, 'name': 'Family'},
  {'id': 14, 'name': 'Fantasy'}],
 'homepage': 'http://www.et20.com/',
 'id': 601,
 'imdb_id': 'tt0083866',
 'original_language': 'en',
 'original_title': 'E.T. the Extra-Terrestrial',
 'overview': 'After a gentle alien becomes stranded on Earth, the being is discovered and befriended by a young boy named Elliott. Bringing the extraterrestrial into his suburban California house, Elliott introduces E.T., as the alien is dubbed, to his brother and his little sister, Gertie, and the children decide to keep its existence a secret. Soon, however, E.T. falls ill, resulting in government intervention and a dire situation for both Elliott and the alien.',
 'popularity': 30.482,
 'poster_path': '/an0nD6uq6byfxXCfk6lQBzdL2J1.jpg',
 'production_companies'

In [20]:
import os 
folder= 'movies/'
os.makedirs(folder, exist_ok=True)
os.listdir(folder)

['.ipynb_checkpoints',
 'akas.csv.gz',
 'basics.csv.gz',
 'final2001.csv.gz',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'ratings.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json']

In [21]:
YEARS_TO_GET= [2000]
YEARS_TO_GET


[2000]

In [22]:
def funciton_movie(movie_id):
# Get the movie object for the current id
    movie = tmbd.Movies(movie_id)
    # save the .info .releases dictionaries
    info = movie.info()
    releases = movie.releases()
    # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1' ] =='US':
            ## save a "certification" key in the info dict with the certification
            info['certification'] = c['certification']
    return info


In [23]:
funciton_movie(602)

{'adult': False,
 'backdrop_path': '/uw4SnKFZ453Gxmj5XR5Susj8TNo.jpg',
 'belongs_to_collection': {'id': 304378,
  'name': 'Independence Day Collection',
  'poster_path': '/AuBMbv3gSAdTaEIzcUySRETIauY.jpg',
  'backdrop_path': '/ou70x2658dVnbycIRaC37sEeafS.jpg'},
 'budget': 75000000,
 'genres': [{'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'},
  {'id': 878, 'name': 'Science Fiction'}],
 'homepage': '',
 'id': 602,
 'imdb_id': 'tt0116629',
 'original_language': 'en',
 'original_title': 'Independence Day',
 'overview': "On July 2, a giant alien mothership enters orbit around Earth and deploys several dozen saucer-shaped 'destroyer' spacecraft that quickly lay waste to major cities around the planet. On July 3, the United States conducts a coordinated counterattack that fails. On July 4, a plan is devised to gain access to the interior of the alien mothership in space, in order to plant a nuclear missile.",
 'popularity': 50.504,
 'poster_path': '/p0BPQGSPoSa8Ml0DAf2mB2kCU0R

In [24]:
#USing a function to append new results to the existing JSON file

def write_json (new_data, filename):
    with open(filename, "r+") as file:
        #First we load existing data into a dict
        file_data = json.load(file)
        if (type(new_data)==list)& (type(file_data)==list):
            file_data.extend(new_data)# can use .append as alternative
        else:
            file_data.append(new_data)
        #setting the current files's position at offset.
        file.seek(0)
        #converting back to json
        json.dump(file_data, file)

In [25]:
#Loading the dataframe from project part 1 as basics
basics = pd.read_csv("movies/basics.csv.gz")
basics.head(5)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0093119,movie,Grizzly II: Revenge,Grizzly II: The Predator,0,2020,NaN,74,"Horror,Music,Thriller"


In [26]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET,desc='YEARS',position=0):
    
    #Defining the JSON file to store results for year
    JSON_FILE = f"{folder}tmdb_api_results_{YEAR}.json"

        #checking for existence of file
    file_exists =os.path.isfile(JSON_FILE)

    #Checking if file already exists, if it does not exist: create it
    if file_exists ==False:

    #save an empty dict with just "imdb_id" to the new json file.

        with open(JSON_FILE, "w") as f:
            json.dump([{"imdb_id":0}], f)

    #Defining and filtering the IDs to call

    #Saving new year as the current df
    df =basics.loc[basics["startYear"]==YEAR].copy()

    #Saving movie ids to the list
    movie_ids =df["tconst"].copy()#.to_list
   #Load in any existing API results with pd.read_json
    #Check to see if any of the movie_ids to get are already in the JSON file.
    #Filter out only movies that are missing from the JSON file to use in the loop

    previous_df = pd.read_json(JSON_FILE)
    

    #filtering movie IDS already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df["imdb_id"])]
   

    
    #INNER LOOP:  .

    #REtrieve the movie ID from list
    for movie_id in tqdm_notebook(movie_ids_to_get, 
                                 desc =f'Movies from {YEAR}',
                                  position =1,
                                  leave =True):

        #retrieve data for the movie id
        try:
            temp =movie_with_rating(movie_id)#This will use the pre-made function
            #Append the results to existing file using the pre-made function
            write_json(temp, JSON_FILE)
            #Set to 20ms sleep to avoid overwhelming server
            time.sleep(0.02)

        #if it fails, make a dict with just the id and None for certification
        except Exception as e:
            continue
        
    final_year_df =pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{folder}final_tmdb_data_{YEAR}.csv.gz", compression ="gzip", index =False)

YEARS:   0%|          | 0/1 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1404 [00:00<?, ?it/s]

In [27]:
file= pd.read_json("movies/tmdb_api_results_2000.json")

In [28]:
for YEAR in tqdm_notebook(YEARS_TO_GET,desc='YEARS',position=0):
    JSON_FILE = f"{folder}tmdb_api_results_{YEAR}.json"
    print(JSON_FILE)
    final_year_df =pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{folder}final_tmdb_data_{YEAR}.csv.gz", compression ="gzip", index =False)

YEARS:   0%|          | 0/1 [00:00<?, ?it/s]

movies/tmdb_api_results_2000.json
